### Backtracking algorithm

In [570]:
# function used in AIMA book code
def flatten(seqs):
    return sum(seqs, [])

In [571]:
def is_valid(grid, row, col, num):
    # create lists for rows, cols, boxes
    rows = flatten([list(map(flatten, brow)) for brow in grid])
    cols = [list(col) for col in zip(*rows)]
    boxes = flatten([list(map(flatten, zip(*brow))) for brow in grid])
    # create box index
    # (3 x row block number) + column block number
    # block: 3 rows or cols together
    box = (3*(row//3))+(col//3)
    # check if num is already used in the row
    for i in range(9):
        if rows[row][i] == num:
            return False
    # check if num is already used in the column
    for i in range(9):
        if cols[col][i] == num:
            return False
    # check if num is already used in the 3x3 box
    for i in range(9):
        if boxes[box][i] == num:
            return False
    return True

In [572]:
def find_empty_space(grid):
    # create list of rows
    rows = flatten([list(map(flatten, brow)) for brow in grid])
    # for each row and col if cell = 0 return cell coordinates
    for row in range(9):
        for col in range(9):
            if rows[row][col] == 0:
                return row, col
    return None

In [573]:
def backtracking(grid, counter=0, backtrack_counter=0):
    # find first empty cell
    empty_cell = find_empty_space(grid)
    # if there are no empty cells left, the puzzle is solved
    if empty_cell is None:  
        print('cell assignments: ',counter)
        print('num of backtracks: ',backtrack_counter)
        return grid
    else:
        # coordinates of empty cell
        row, col = empty_cell
        # find row and col block of empty cell
        row_block = row // 3
        col_block = col // 3
        # for each number in domain (1-9)
        for num in range(1, 10):
            # if the number fits in coordinates
            if is_valid(grid, row, col, num):
                # cell = num
                counter += 1
                grid[row_block][row % 3][col_block][col % 3] = num
                # recursively call solve with the updated grid
                result = backtracking(grid, counter, backtrack_counter)
                if result is not None:
                    return result
                # if not valid (backtrack) set cell to 0
                grid[row_block][row % 3][col_block][col % 3] = 0
                backtrack_counter += 1
    # if no solution is found return none -> goes to above line (set cell to 0)
    return None

In [574]:
# to convert input string into grid
def string_to_grid(puzzle_str):
    # empty grid to fill in 
    grid = [
        [[[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]]],
        [[[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]]],
        [[[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]]]]

    for rowblock in range(3):
        for row in range(3):
            for triple in range(3):
                for cell in range(3):
                    rown = (rowblock*3)+row
                    coln = (triple*3)+cell
                    celln = rown*9+coln
                    value = puzzle_str[celln]
                    if value == '.':
                        value = 0
                    else:
                        value = int(value)
                    grid[rowblock][row][triple][cell] = value
    return grid

### AC-3 algorithm

In [575]:
def create_grid():
    # create sudoku grid with coordinates
    coords = []
    for r in range(9):
        for c in range(9):
            coords += [(r,c)]

    coord_grid = [
        [[[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]]],
        [[[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]]],
        [[[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]],
        [[0,0,0],[0,0,0],[0,0,0]]]]

    for rowblock in range(3):
        for row in range(3):
            for triple in range(3):
                for cell in range(3):
                    rown = (rowblock*3)+row
                    coln = (triple*3)+cell
                    celln = rown*9+coln
                    coord_grid[rowblock][row][triple][cell] = coords[celln]
    return coord_grid

In [576]:
def create_constraints():
    # creates list of basic sudoku constraints
    # all cells in same rows must be different
    # all cells in same columns must be different
    # all cells in same boxes must be different
    
    coord_grid = create_grid()

    # create lists for rows, cols, boxes
    rows_coord = flatten([list(map(flatten, brow)) for brow in coord_grid])
    cols_coord = [list(col) for col in zip(*rows_coord)]
    boxes_coord = flatten([list(map(flatten, zip(*brow))) for brow in coord_grid])

    # create constraints
    
    row_constraints = {}
    for row in rows_coord:
        for i in range(9):
            for j in range(i+1, 9):
                coord1 = f"{row[i]}"
                coord2 = f"{row[j]}"
                row_constraints[(coord1, coord2)] = lambda x, y: x != y
#                 row_constraints[(coord2, coord1)] = lambda x, y: x != y

    col_constraints = {}
    for col in cols_coord:
        for i in range(9):
            for j in range(i+1, 9):
                coord1 = f"{col[i]}"
                coord2 = f"{col[j]}"
                col_constraints[(coord1, coord2)] = lambda x, y: x != y
#                 col_constraints[(coord2, coord1)] = lambda x, y: x != y


    box_constraints = {}
    for box in boxes_coord:
        for i in range(9):
            for j in range(i+1, 9):
                coord1 = f"{box[i]}"
                coord2 = f"{box[j]}"
                box_constraints[(coord1, coord2)] = lambda x, y: x != y
#                 box_constraints[(coord2, coord1)] = lambda x, y: x != y

    constraints = {}
    constraints.update(row_constraints)
    constraints.update(col_constraints)
    constraints.update(box_constraints)
    return constraints

In [577]:
def reverse_tuple(tuples):
    new_tup = tuples[::-1]
    return new_tup

In [578]:
def create_domains(grid):
    # create domains for all cells
    # any empty cell will have a domain of 1,2,3,4,5,6,7,8,9
    # any cell that has a value will have a domain of that value
    domains = {}
    rows = flatten([list(map(flatten, brow)) for brow in grid])

    for ir, r in enumerate(rows):
        for ic, value in enumerate(r):
            if value == 0:
                coord = f"({ir}, {ic})"
                domains[coord] = {1, 2, 3, 4, 5, 6, 7, 8, 9}
            else:
                coord = f"({ir}, {ic})"
                domains[coord] = {value}
    return domains

In [579]:
# AC-3
# 1. Turn each constraint into 2 ex: (A!= B) & (B != A)
# 2. Add all arcs to agenda (included in constraints)
# 3. Repeat below until agenda (constraints) is empty
# - take an arc (xi,xj) off the agenda and check it
# - for EVERY value of xi there must be ANY value of xj that satisfies it
# - remove any inconsistent value from xi
# - if xi has changed add all arcs of the form (xk,xi) to the agenda

def ac3(constraints, domains):
    full_constraints = constraints.copy()
    # create dict with reverse constraints
    reverse_constraints = {reverse_tuple(key): value for key, value in full_constraints.items()}
    # update constraints with reverse_constraints
    full_constraints.update(reverse_constraints)
    # to avoid changing original constraints and domains make a copy
    c = full_constraints.copy()
    d = domains.copy()
    while c:
        # (xi,xj) first pair of coordinates from constraints
        (xi,xj) = list(c.keys())[0]
        # pops the constraint function related to those coordinates
        f = c.pop((xi,xj)) 
        # for EVERY value of xi there must be ANY value of xj that satisfies it
        for x in d[xi].copy():
            if not any(f(x, y) for y in d[xj]):
                # if there is no value xj that satisfies any x from xi, remove x from domain of xi
                d[xi].remove(x)
                # if xi has changed all any constraints of the form (xk,xi)
                # match original constraints where xi is the second element of the coordinates
                add_constraints_keys = list(filter(lambda k: k[1] == xi, full_constraints.keys()))
                # create dict with constraints we need to add back
                add_constraints = {key: value for key, value in full_constraints.items() if key in add_constraints_keys}
                # update c with add_constraints
                c.update(add_constraints)
    return d

In [580]:
def domain_to_grid(ac3):
    coord_grid = create_grid()
    for i in range(3):
        for j in range(len(coord_grid[i])):
            for k in range(len(coord_grid[i][j])):
                for z in range(len(coord_grid[i][j][k])):
                    coord = coord_grid[i][j][k][z]
                    value = ac3[str(coord)] # domain of coordinate
                    coord_grid[i][j][k][z] = list(value)[0] # first item in domain
    return coord_grid

### Local Search algorithm

function MIN-CONFLICTS(csp,max steps) returns a solution or failure
    inputs: 
        csp, a constraint satisfaction problem
        max_steps, the number of steps allowed before giving up
        
    current ← an initial complete assignment 
    for i =1 to max_steps do
        if current is a solution for csp then return current
        var ← a randomly chosen conflicted variable from csp.VARIABLES
        value ←the value v for var that minimizes CONFLICTS(var,v,current,csp) 
        set var = value in current
    return failure

The MIN-CONFLICTS algorithm for solving CSPs by local search. The initial state may be chosen randomly or by a greedy assignment process that chooses a minimal- conflict value for each variable in turn. The C O N FL I C T S function counts the number of constraints violated by a particular value, given the rest of the current assignment.

In [ ]:
# fill grid with random values

In [558]:
constraints[('(0, 0)','(0, 1)')]

<function __main__.create_constraints.<locals>.<lambda>(x, y)>

In [ ]:
# book code
def min_conflicts(csp, max_steps=100000):
    """Solve a CSP by stochastic Hill Climbing on the number of conflicts."""
    # Generate a complete assignment for all variables (probably with conflicts)
    csp.current = current = {}
    for var in csp.variables:
        val = min_conflicts_value(csp, var, current)
        csp.assign(var, val, current)
    # Now repeatedly choose a random conflicted variable and change it
    for i in range(max_steps):
        conflicted = csp.conflicted_vars(current)
        if not conflicted:
            return current
        var = random.choice(conflicted)
        val = min_conflicts_value(csp, var, current)
        csp.assign(var, val, current)
    return None
def min_conflicts_value(csp, var, current):
    """Return the value that will give var the least number of conflicts.
    If there is a tie, choose at random."""
    return argmin_random_tie(csp.domains[var], key=lambda val: csp.nconflicts(var, val, current))

def nconflicts(self, var, val, assignment):
        """The number of conflicts, as recorded with each assignment.
        Count conflicts in row and in up, down diagonals. If there
        is a queen there, it can't conflict with itself, so subtract 3."""
        n = len(self.variables)
        c = self.rows[val] + self.downs[var + val] + self.ups[var - val + n - 1]
        if assignment.get(var, None) == val:
            c -= 3
        return c


# Sudoku Puzzle

In [551]:
easy1 = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
harder1 = '417369805030000000000700000020000060000080400000010000000603070500200000104000000'

In [552]:
grid1 = string_to_grid(easy1)
grid2 = string_to_grid(harder1)

In [553]:
grid1

[[[[0, 0, 3], [0, 2, 0], [6, 0, 0]],
  [[9, 0, 0], [3, 0, 5], [0, 0, 1]],
  [[0, 0, 1], [8, 0, 6], [4, 0, 0]]],
 [[[0, 0, 8], [1, 0, 2], [9, 0, 0]],
  [[7, 0, 0], [0, 0, 0], [0, 0, 8]],
  [[0, 0, 6], [7, 0, 8], [2, 0, 0]]],
 [[[0, 0, 2], [6, 0, 9], [5, 0, 0]],
  [[8, 0, 0], [2, 0, 3], [0, 0, 9]],
  [[0, 0, 5], [0, 1, 0], [3, 0, 0]]]]

In [554]:
grid2

[[[[4, 1, 7], [3, 6, 9], [8, 0, 5]],
  [[0, 3, 0], [0, 0, 0], [0, 0, 0]],
  [[0, 0, 0], [7, 0, 0], [0, 0, 0]]],
 [[[0, 2, 0], [0, 0, 0], [0, 6, 0]],
  [[0, 0, 0], [0, 8, 0], [4, 0, 0]],
  [[0, 0, 0], [0, 1, 0], [0, 0, 0]]],
 [[[0, 0, 0], [6, 0, 3], [0, 7, 0]],
  [[5, 0, 0], [2, 0, 0], [0, 0, 0]],
  [[1, 0, 4], [0, 0, 0], [0, 0, 0]]]]

# Backtracking Algorithm

In [305]:
backtracking_grid1 = backtracking(grid1)

cell assignments:  59
num of backtracks:  10


In [306]:
backtracking_grid2 = backtracking(grid2)

cell assignments:  74
num of backtracks:  15


In [307]:
backtracking_grid1

[[[[4, 8, 3], [9, 2, 1], [6, 5, 7]],
  [[9, 6, 7], [3, 4, 5], [8, 2, 1]],
  [[2, 5, 1], [8, 7, 6], [4, 9, 3]]],
 [[[5, 4, 8], [1, 3, 2], [9, 7, 6]],
  [[7, 2, 9], [5, 6, 4], [1, 3, 8]],
  [[1, 3, 6], [7, 9, 8], [2, 4, 5]]],
 [[[3, 7, 2], [6, 8, 9], [5, 1, 4]],
  [[8, 1, 4], [2, 5, 3], [7, 6, 9]],
  [[6, 9, 5], [4, 1, 7], [3, 8, 2]]]]

In [293]:
backtracking_grid2 = backtracking(grid2)

cell assignments:  74
num of backtracks:  15


In [308]:
backtracking_grid2

[[[[4, 1, 7], [3, 6, 9], [8, 2, 5]],
  [[6, 3, 2], [1, 5, 8], [9, 4, 7]],
  [[9, 5, 8], [7, 2, 4], [3, 1, 6]]],
 [[[8, 2, 5], [4, 3, 7], [1, 6, 9]],
  [[7, 9, 1], [5, 8, 6], [4, 3, 2]],
  [[3, 4, 6], [9, 1, 2], [7, 5, 8]]],
 [[[2, 8, 9], [6, 4, 3], [5, 7, 1]],
  [[5, 7, 3], [2, 9, 1], [6, 8, 4]],
  [[1, 6, 4], [8, 7, 5], [2, 9, 3]]]]

# AC-3 Algorithm

In [555]:
constraints = create_constraints()
domains = create_domains(grid1)
ac3 = ac3(constraints, domains)
domain_to_grid(ac3)

[[[[4, 8, 3], [9, 2, 1], [6, 5, 7]],
  [[9, 6, 7], [3, 4, 5], [8, 2, 1]],
  [[2, 5, 1], [8, 7, 6], [4, 9, 3]]],
 [[[5, 4, 8], [1, 3, 2], [9, 7, 6]],
  [[7, 2, 9], [5, 6, 4], [1, 3, 8]],
  [[1, 3, 6], [7, 9, 8], [2, 4, 5]]],
 [[[3, 7, 2], [6, 8, 9], [5, 1, 4]],
  [[8, 1, 4], [2, 5, 3], [7, 6, 9]],
  [[6, 9, 5], [4, 1, 7], [3, 8, 2]]]]

In [533]:
grid1

[[[[0, 0, 3], [0, 2, 0], [6, 0, 0]],
  [[9, 0, 0], [3, 0, 5], [0, 0, 1]],
  [[0, 0, 1], [8, 0, 6], [4, 0, 0]]],
 [[[0, 0, 8], [1, 0, 2], [9, 0, 0]],
  [[7, 0, 0], [0, 0, 0], [0, 0, 8]],
  [[0, 0, 6], [7, 0, 8], [2, 0, 0]]],
 [[[0, 0, 2], [6, 0, 9], [5, 0, 0]],
  [[8, 0, 0], [2, 0, 3], [0, 0, 9]],
  [[0, 0, 5], [0, 1, 0], [3, 0, 0]]]]

In [535]:
ac3 = ac3(constraints, domains)

TypeError: 'dict' object is not callable

In [517]:
constraints = create_constraints()
domains = create_domains(grid2)
ac3 = ac3(constraints, domains)
domain_to_grid(ac3)

TypeError: 'dict' object is not callable

In [472]:
grid2

[[[[4, 1, 7], [3, 6, 9], [8, 0, 5]],
  [[0, 3, 0], [0, 0, 0], [0, 0, 0]],
  [[0, 0, 0], [7, 0, 0], [0, 0, 0]]],
 [[[0, 2, 0], [0, 0, 0], [0, 6, 0]],
  [[0, 0, 0], [0, 8, 0], [4, 0, 0]],
  [[0, 0, 0], [0, 1, 0], [0, 0, 0]]],
 [[[0, 0, 0], [6, 0, 3], [0, 7, 0]],
  [[5, 0, 0], [2, 0, 0], [0, 0, 0]],
  [[1, 0, 4], [0, 0, 0], [0, 0, 0]]]]